# Seleção das estações e poluentes
As estações tem medições horárias.

Para que um dia seja considerado válido **para uma estação e um poluente** é necessário que tenha pelo menos 75% das obervações, ou seja pelo menos 18 observações.

Para que um ano seja considerado válido **para uma estação e um poluente** é necessário que tenha pelo menos 75% dos dias válidos ou seja pelo menos 328 dias.

Para que um poluente de uma estação seja considerado para as análises posteriores tem que ter pelo menos 2 anos válidos

## Inicialização

In [1]:
import numpy as np
import pandas as pd

C:\Users\guilh\AppData\Local\Temp\ipykernel_20324\1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Diretoria onde os dados estão localizados
dir_files = './03-dados-qualar-longo-corrigido/'

In [3]:
# Ficheiro das medições 
ficheiro_medicoes = dir_files + '03-medicoes-longo-AML.csv'

In [4]:
medicoes = pd.read_csv(ficheiro_medicoes, thousands=',', index_col=0, parse_dates=True)
medicoes

,estacao,poluente,valor
index,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6
...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1


In [5]:
# Criar uma coluna com apenas o dia da data
medicoes['Dia']= medicoes.index.day
# Criar uma coluna com o ano da data
medicoes['Ano']=medicoes.index.year
# Conferir resultados
medicoes

,estacao,poluente,valor,Dia,Ano
index,,,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9,1,2001
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9,1,2001
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6,1,2001
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6,1,2001
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6,1,2001
...,...,...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8,31,2022
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5,31,2022
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1,31,2022


## Cálculo de horas por dia e dias válidos
Vamos analisar a quantidade de dias válidos presentes nos dados.

In [6]:
# Cacular horas por dia

# Criando uma nova coluna 'Hora' baseada no índice
medicoes['Hora'] = medicoes.index.hour

medicoes_validas = medicoes[medicoes['valor'] > 0]

# Agrupando por data, estação e poluente e contando as horas
horas_por_dia = medicoes_validas.groupby([medicoes_validas.index.date, 'estacao', 'poluente'])['Hora'].count()
# Convertendo a série em um DataFrame
horas_por_dia = horas_por_dia.reset_index()
horas_por_dia.columns = ['Data', 'Estacao', 'Poluente', 'medições']

horas_por_dia

,Data,Estacao,Poluente,medições
0,2001-01-01,Alfragide/Amadora,NO2,24
1,2001-01-01,Alfragide/Amadora,O3,24
2,2001-01-01,Alfragide/Amadora,SO2,24
3,2001-01-01,Avenida da Liberdade,NO2,24
4,2001-01-01,Avenida da Liberdade,PM10,22
...,...,...,...,...
443518,2022-12-31,Reboleira,PM10,24
443519,2022-12-31,Restelo,NO2,24
443520,2022-12-31,Restelo,O3,24
443521,2022-12-31,Santa Cruz de Benfica,NO2,24


In [7]:
# Calcular dias válidos
validos=[]
for i in horas_por_dia['medições']:
    if i>=18:
        validos.append(1)  #uma maneira que encontramos para mostrar todos os dias válidos
    else:
        validos.append(0)

horas_por_dia['Dias Válidos']=validos
horas_por_dia.drop(columns='medições', inplace=True)
horas_por_dia
    

,Data,Estacao,Poluente,Dias Válidos
0,2001-01-01,Alfragide/Amadora,NO2,1
1,2001-01-01,Alfragide/Amadora,O3,1
2,2001-01-01,Alfragide/Amadora,SO2,1
3,2001-01-01,Avenida da Liberdade,NO2,1
4,2001-01-01,Avenida da Liberdade,PM10,1
...,...,...,...,...
443518,2022-12-31,Reboleira,PM10,1
443519,2022-12-31,Restelo,NO2,1
443520,2022-12-31,Restelo,O3,1
443521,2022-12-31,Santa Cruz de Benfica,NO2,1


In [8]:
# Mostrar número de dias válidos/não válidos nos dados
dias_validos=horas_por_dia['Dias Válidos'].sum()
dias_nao_validos= horas_por_dia.shape[0]-dias_validos

print(dias_validos)
print(dias_nao_validos)

396497
47026


## Cálculo dos dias válidos por ano e anos válidos
Vamos analisar a quantidade de anos válidos presentes nos dados.

In [9]:
# Preparar dataframe com informação de dias válidos e ano
horas_por_dia.set_index(horas_por_dia['Data'], inplace=True)
horas_por_dia.drop(columns='Data', inplace=True)             #esta não é mais necessária pois já está como index
horas_por_dia.index = pd.to_datetime(horas_por_dia.index)    #certificar que está como data
horas_por_dia['Ano'] = horas_por_dia.index.year
horas_por_dia


,Estacao,Poluente,Dias Válidos,Ano
Data,,,,
2001-01-01,Alfragide/Amadora,NO2,1,2001
2001-01-01,Alfragide/Amadora,O3,1,2001
2001-01-01,Alfragide/Amadora,SO2,1,2001
2001-01-01,Avenida da Liberdade,NO2,1,2001
2001-01-01,Avenida da Liberdade,PM10,1,2001
...,...,...,...,...
2022-12-31,Reboleira,PM10,1,2022
2022-12-31,Restelo,NO2,1,2022
2022-12-31,Restelo,O3,1,2022


In [10]:
# Calcular dias válidos por ano
dias_validos_ano=horas_por_dia.groupby(['Estacao','Poluente', 'Ano']).sum()
dias_validos_ano

Dias Válidos
Estacao               Poluente Ano               
Alfragide/Amadora     NO2      2001           346
                               2002           342
                               2003           339
                               2004           274
                               2005           338
...                                           ...
Santa Cruz de Benfica SO2      2008             5
                               2009           288
                               2010            75
                               2011            57
                               2012            62

[1388 rows x 1 columns]

In [11]:
# Calcular anos válidos por estação e poluente
dias_validos_ano['ano válido']=dias_validos_ano['Dias Válidos']>328
dias_validos_ano

Dias Válidos  ano válido
Estacao               Poluente Ano                           
Alfragide/Amadora     NO2      2001           346        True
                               2002           342        True
                               2003           339        True
                               2004           274       False
                               2005           338        True
...                                           ...         ...
Santa Cruz de Benfica SO2      2008             5       False
                               2009           288       False
                               2010            75       False
                               2011            57       False
                               2012            62       False

[1388 rows x 2 columns]

In [12]:
# Mostrar número de anos válidos
so_anos_válidos=dias_validos_ano[dias_validos_ano['Dias Válidos']>328]
so_anos_válidos.count()

Dias Válidos    832
ano válido      832
dtype: int64

## Medições de poluentes e estações válidas
Vamos manter apenas as medições correspondentes a anos válidos.

In [13]:
medicoes2=medicoes.copy()
#juntar o dataframe inicial deste caderno com o dataframe com a informação dos dias e anos válidos
medicoes2 = medicoes2.merge(so_anos_válidos.reset_index(), how='left',left_on=['estacao', 'poluente', 'Ano'],right_on=['Estacao', 'Poluente', 'Ano'])
medicoes2['ano válido'] = medicoes2['ano válido'].fillna(False).astype(bool) #Substituimos os valores nulos por "False" e certificamo-nos que está como booleano e não como string
medicoes2.set_index(medicoes.index, inplace=True)
medicoes2.drop(['Estacao','Poluente','Dias Válidos'], axis=1, inplace=True) #como ficamos com colunas duplicadas do merge vamos removê-las aqui

medicoes=medicoes2.copy()

medicoes

C:\Users\guilh\AppData\Local\Temp\ipykernel_20324\587361559.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  medicoes2['ano válido'] = medicoes2['ano válido'].fillna(False).astype(bool) #Substituimos os valores nulos por "False" e certificamo-nos que está como booleano e não como string


,estacao,poluente,valor,Dia,Ano,Hora,ano válido
index,,,,,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9,1,2001,0,False
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1,False
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6,1,2001,2,False
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6,1,2001,3,False
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6,1,2001,4,False
...,...,...,...,...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8,31,2022,19,True
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5,31,2022,20,True
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1,31,2022,21,True


In [14]:
# Manter apenas as medições de anos válidos
medicoes=medicoes[medicoes['ano válido']==True]
medicoes

,estacao,poluente,valor,Dia,Ano,Hora,ano válido
index,,,,,,,
2007-01-01 00:00:00,Alfragide/Amadora,SO2,6.9,1,2007,0,True
2007-01-01 01:00:00,Alfragide/Amadora,SO2,3.0,1,2007,1,True
2007-01-01 02:00:00,Alfragide/Amadora,SO2,4.6,1,2007,2,True
2007-01-01 03:00:00,Alfragide/Amadora,SO2,3.1,1,2007,3,True
2007-01-01 04:00:00,Alfragide/Amadora,SO2,2.5,1,2007,4,True
...,...,...,...,...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8,31,2022,19,True
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5,31,2022,20,True
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1,31,2022,21,True


In [15]:
# Calcular medições válidas por estação

medicoes.groupby('estacao')['ano válido'].sum()

estacao
Alfragide/Amadora               230417
Alto Seixalinho                  59051
Alverca                         355889
Avenida da Liberdade            373836
Beato                           314052
Benfica                          59059
Cascais - Escola da Cidadela     93031
Cascais-Mercado                 161618
Chelas                           76711
Câmara Municipal                  8723
Entrecampos                     557671
Escavadeira                     417799
Fidalguinhos                     42404
Laranjeiro                      510799
Lavradio                        262278
Loures-Centro                   364151
Mem Martins                     575083
Odivelas-Ramada                 263952
Olivais                         587968
Paio Pires                      475239
Quinta do Marquês               425220
Reboleira                       357163
Restelo                         330790
Santa Cruz de Benfica           143142
Name: ano válido, dtype: int64

In [16]:
# Calcular medições válidas por poluente

medicoes.groupby('poluente')['ano válido'].sum()

poluente
NO2      2528155
O3       1851718
PM10     1914902
PM2.5     458645
SO2       292626
Name: ano válido, dtype: int64